In [1]:
import tensorflow as tf
physical_devices = tf.config.list_physical_devices()
print(physical_devices)
tf.config.experimental.set_memory_growth(physical_devices[1], True)

2022-04-13 17:39:08.096018: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2022-04-13 17:39:10.322960: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-04-13 17:39:10.326938: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-04-13 17:39:10.408034: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-13 17:39:10.408453: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce GTX 1650 computeCapability: 7.5
coreClock: 1.56GHz coreCount: 16 deviceMemorySize: 3.81GiB deviceMemoryBandwidth: 119.24GiB/s
2022-04-13 17:39:10.408491: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2022-04-13 17:39:10.435785: I tensorflow/stream_executor/platform/def

In [2]:
from tensorflow.keras.layers import Dense, Input, Lambda, Flatten, Dropout
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
import tensorflow_probability as tfp
from collections import deque
import os
import gym
import numpy as np
import wandb
import random
import math
from tensorflow.keras.losses import KLDivergence
import time

In [3]:
import datetime
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
# rm -rf ./logs/train_PPO/
train_log_dir = "logs/train_PPO/" + current_time
train_summary_writer = tf.summary.create_file_writer(train_log_dir)
%load_ext tensorboard
%tensorboard --logdir {train_log_dir}

2022-04-13 17:39:11.362321: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-04-13 17:39:11.362967: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-13 17:39:11.363177: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce GTX 1650 computeCapability: 7.5
coreClock: 1.56GHz coreCount: 16 deviceMemorySize: 3.81GiB deviceMemoryBandwidth: 119.24GiB/s
2022-04-13 17:39:11.363210: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libc

In [4]:
env_name = "CartPole-v1"
total_timesteps = 25_000
init_lr = 2.5e-4
seed = 1
num_envs = 4
num_steps = 128
batch_size = num_envs * num_steps
anneal_lr = True
gamma = 0.99
lamda = 0.95
gae = True
num_minibatch = 4
minibatch_size = batch_size//num_minibatch
update_epochs = 4
norm_adv = True
clip_coeff = 0.2
clip_value_loss = True
entropy_coeff = 0.01
valueloss_coeff = 0.5
max_grad_norm = 0.5
target_kl = None

In [5]:
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)

In [40]:
env = gym.make(env_name)
env = gym.wrappers.RecordEpisodeStatistics(env)
env = gym.wrappers.RecordVideo(env, "videos", step_trigger=lambda t: t%100==0)
env.seed(seed)

observation = env.reset()
input_size = len(observation)
output_size = env.action_space.n

/home/himanshu/anaconda3/envs/myenv/lib/python3.9/site-packages/gym/wrappers/record_video.py:41: UserWarning: WARN: Overwriting existing videos at /home/himanshu/Documents/my_project5/RLandNEAT/PPO/videos folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


In [7]:
input_size, output_size

(4, 2)

In [8]:
print(f"Observations Shape: {env.observation_space.shape}")
print(f"Action Shape: {env.action_space.shape}")

Observations Shape: (4,)
Action Shape: ()


In [9]:
class Agent:
    def __init__(self):
        self.actor = self.build_actor()
        self.critic = self.build_critic()
        
    def build_critic(self):
        inputs = Input(shape = input_size)
        x = Dense(64, activation = 'tanh')(inputs)
        x = Dense(64, activation = 'tanh')(x)
        x = Dense(1, activation = 'linear')(x)
        model = Model(inputs = inputs, outputs = x)
        return model
    
    def build_actor(self):
        inputs = Input(shape = input_size)
        x = Dense(64, activation = 'tanh')(inputs)
        x = Dense(64, activation = 'tanh')(x)
        x = Dense(output_size, activation = 'linear')(x)
        model = Model(inputs = inputs, outputs = x)
        return model
    
    def get_value(self, x):
        return self.critic(x)
    
    def get_actor_critic_value(self, x, action = None):
        logits = self.actor(x)
        dis = tfp.distributions.Categorical(logits=logits)
        if action is None:
            action = dis.sample()
        
        return np.squeeze(action), dis.log_prob(action), dis.entropy(), tf.squeeze(self.critic(x))
    

In [10]:
agent = Agent()

In [11]:
states = np.zeros(shape = (num_steps, num_envs) + env.observation_space.shape, dtype = np.float32)
actions = np.zeros(shape = ((num_steps, num_envs)+ env.action_space.shape), dtype = np.float32)
log_probs = np.zeros(shape = (num_steps, num_envs), dtype = np.float32)
rewards = np.zeros(shape = (num_steps, num_envs), dtype = np.float32)
dones = np.zeros(shape = (num_steps, num_envs), dtype = np.float32)
values = np.zeros(shape = (num_steps, num_envs), dtype = np.float32)


global_step = 0
start_time = time.time()
next_states = np.array([env.reset() for _ in range(num_envs)])
next_dones = np.zeros(num_envs)
num_updates = total_timesteps//batch_size
optimizer = Adam(epsilon = 1e-5)

for update in range(1, num_updates + 1):
    if anneal_lr:
        frac = 1.0 - (update - 1.0)/num_updates
        lr_now = frac * init_lr
        optimizer.lr = lr_now
    
    for e in range(num_envs):

        
        next_state = np.array(env.reset())
        done = False
        for step in range(num_steps):
            global_step += 1 
            states[step, e] = next_states[e]
            dones[step, e] =  next_dones[e]
            action, log_prob, _, value = agent.get_actor_critic_value(np.expand_dims(next_state, axis=0))
            actions[step, e] = action
            log_probs[step, e] = log_prob
            values[step, e] = value
            
            next_state, reward, done, info = env.step(action)
            rewards[step, e] = reward

            next_states[e] = next_state
            next_dones[e] = done
            
            if "episode" in info.keys():
                episodic_return = info["episode"]["r"]
                print(f"Global Step={global_step}, Episodic_return={episodic_return}")
                with train_summary_writer.as_default(step = global_step):
                    tf.summary.scalar("Charts/episodic_return", episodic_return, global_step)
                    tf.summary.scalar("Charts/episodic_length", info["episode"]["l"], global_step)
            
            
        
            if done and step < num_steps-1:
                next_state = np.array(env.reset())
                done = False
        
        
        
    next_value = np.squeeze(agent.get_value(next_states))
    if gae:
        advantages = np.zeros_like(rewards)
        last_gae = 0.0
        for step in reversed(range(num_steps)):
            if step == num_steps - 1:
                next_non_terminal = 1.0 - next_dones
                next_values = next_value
            else:
                next_non_terminal = 1.0 - dones[step+1]
                next_values = values[step+1]
                
            delta = rewards[step] + gamma * next_values * next_non_terminal - values[step]     
            advantages[step] = last_gae = delta + gamma * lamda * next_non_terminal * last_gae
        returns = advantages + values
    else:
        returns = np.zeros_like(rewards)
        for step in reversed(range(num_steps)):
            if step == num_steps -1:
                next_non_terminal  = 1.0 - next_dones
                next_return = next_value
            else:
                next_non_terminal = 1.0 - dones[step+1]
                next_return = returns[step+1]
            returns[step] = rewards[step] + gamma * next_non_terminal * next_return
        advantages = returns - values
        
    b_states = states.reshape((-1,)+ env.observation_space.shape)
    b_actions = actions.reshape((-1,)+ env.action_space.shape)
    b_log_probs = log_probs.reshape((-1,))
    b_values = values.reshape((-1,))
    b_advantages = advantages.reshape((-1,))
    b_returns = returns.reshape((-1,))
    
    b_indx = np.arange(batch_size)
    clip_fracs = []
    for epoch in range(update_epochs):
        np.random.shuffle(b_indx)
        for start in range(0, batch_size, minibatch_size):
            end = start + minibatch_size
            mb_indx = b_indx[start:end]
            with tf.GradientTape(persistent = True) as tape:
                
                _, new_log_prob, new_entropy, new_value = agent.get_actor_critic_value(b_states[mb_indx], b_actions[mb_indx])
                
                log_ratio = new_log_prob - b_log_probs[mb_indx]
                ratio = tf.math.exp(log_ratio)
                
                # debugging metrics
                approx_kl = tf.reduce_mean((ratio - 1) - log_ratio)
                clip_fracs += [tf.reduce_mean(tf.cast(tf.abs(ratio - 1.0) > clip_coeff, dtype = tf.float32))]
                
                clipped_ratio = tf.clip_by_value(ratio, 1-clip_coeff, 1+clip_coeff)
                
                mb_advantages = b_advantages[mb_indx]

                if norm_adv:
                    mb_advantages = (mb_advantages - mb_advantages.mean())/(mb_advantages.std() + 1e-8)

                ppo_loss = tf.reduce_mean(tf.math.maximum(-ratio * mb_advantages, -clipped_ratio * mb_advantages))

                if clip_value_loss:
                    v_loss_unclipped = (new_value - b_returns[mb_indx]) ** 2
                    v_clipped = b_values[mb_indx] + tf.clip_by_value(new_value - b_values[mb_indx], -clip_coeff, clip_coeff)
                    v_loss_clipped = (v_clipped - b_returns[mb_indx]) ** 2
                    v_loss = 0.5 * tf.reduce_mean(tf.math.maximum(v_loss_unclipped, v_loss_clipped))
                else:
                    v_loss = 0.5 * tf.reduce_mean((new_value - b_returns[mb_indx]) ** 2)
                

                entropy_loss = tf.reduce_mean(new_entropy)
                loss = ppo_loss + valueloss_coeff * v_loss - entropy_coeff * entropy_loss
            
            actor_grads = tape.gradient(loss, agent.actor.trainable_weights)
            critic_grads = tape.gradient(loss, agent.critic.trainable_weights)
            
            actor_grads = [tf.clip_by_norm(g, max_grad_norm) for g in actor_grads]
            critic_grads = [tf.clip_by_norm(g, max_grad_norm) for g in critic_grads]
            
            optimizer.apply_gradients(zip(actor_grads, agent.actor.trainable_weights))
            optimizer.apply_gradients(zip(critic_grads, agent.critic.trainable_weights))
    
        if target_kl is not None:
            if approx_kl > target_kl:
                break
        
    var_true = np.var(b_returns)
    explained_var = np.nan if var_true == 0 else 1 - np.var(b_returns - b_values)/var_true
    with train_summary_writer.as_default():
        tf.summary.scalar("Charts/learning_rate", optimizer.lr, global_step)
        tf.summary.scalar("losses/value_loss", v_loss, global_step)
        tf.summary.scalar("losses/policy_loss", ppo_loss, global_step)
        tf.summary.scalar("losses/entropy", entropy_loss, global_step)
        tf.summary.scalar("losses/approx_kl", approx_kl, global_step)
        tf.summary.scalar("losses/clipfrac", np.mean(clip_fracs), global_step)
        tf.summary.scalar("losses/explained_variance", explained_var, global_step)
        print("SPS:", int(global_step / (time.time() - start_time)))
        tf.summary.scalar("Charts/SPS", int(global_step / (time.time() - start_time)), global_step)
env.close()
# train_summary_writer.close()

2022-04-13 17:39:15.768720: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10


Global Step=16, Episodic_return=16.0
Global Step=31, Episodic_return=15.0
Global Step=48, Episodic_return=17.0
Global Step=62, Episodic_return=14.0
Global Step=90, Episodic_return=28.0
Global Step=101, Episodic_return=11.0
Global Step=113, Episodic_return=12.0
Global Step=139, Episodic_return=11.0
Global Step=199, Episodic_return=60.0
Global Step=217, Episodic_return=18.0
Global Step=236, Episodic_return=19.0
Global Step=250, Episodic_return=14.0
Global Step=277, Episodic_return=21.0
Global Step=314, Episodic_return=37.0
Global Step=355, Episodic_return=41.0
Global Step=373, Episodic_return=18.0
Global Step=412, Episodic_return=28.0
Global Step=433, Episodic_return=21.0
Global Step=448, Episodic_return=15.0
Global Step=474, Episodic_return=26.0
Global Step=496, Episodic_return=22.0
SPS: 121
Global Step=552, Episodic_return=40.0
Global Step=568, Episodic_return=16.0
Global Step=594, Episodic_return=26.0
Global Step=608, Episodic_return=14.0
Global Step=671, Episodic_return=31.0
Global S

In [12]:
env.observation_space.shape + (-1,)

(4, -1)

In [41]:
next_state = env.reset()
done = False
while not done:
    env.render()
    action, _, _, _ = agent.get_actor_critic_value(np.expand_dims(next_state, axis=0))
    next_state, reward, done, info = env.step(action)
    if "episode" in info.keys():
        episodic_return = info["episode"]["r"]
        l = info["episode"]["l"]
        print(f"Episodic_return={episodic_return}, Episode Length: {l}")
env.close()   

Episodic_return=402.0, Episode Length: 402


In [24]:
env.close()